In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nba_api.stats.endpoints import leaguedashplayerstats 

In [4]:
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = [12,8]
print('Setup Complete')

Setup Complete


In [6]:
print("Fetching General Stats...")

general_stats = leaguedashplayerstats.LeagueDashPlayerStats(
    season='2024-25',
    measure_type_detailed_defense='Base'
).get_data_frames()[0]

Fetching General Stats...


In [9]:
print('Fetching Advanced Stats...')

advanced_stats = leaguedashplayerstats.LeagueDashPlayerStats(
    season='2024-25',
    measure_type_detailed_defense='Advanced'
).get_data_frames()[0]

Fetching Advanced Stats...


In [10]:
df = pd.merge(general_stats, advanced_stats[['PLAYER_ID', 'TS_PCT', 'USG_PCT', 'OFF_RATING', 'DEF_RATING']], on='PLAYER_ID', how='left')

In [13]:
pd.set_option('display.max_columns', None)
df.head()


,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,TEAM_COUNT,TS_PCT,USG_PCT,OFF_RATING,DEF_RATING
0,1630639,A.J. Lawson,A.J.,1610612761,TOR,24.0,26,14,12,0.538,486.410000,80,190,0.421,33,101,0.327,43,63,0.683,20,66,86,31,15,13,6,12,44,38,236,-17,427.7,2,0,424.0,424,364,146,239,379,352,340,388,282,281,319,281,267,419,364,359,368,397,414,397,383,241,200,327,337,299,375,159,44,364,1,0.542,0.189,109.7,111.1
1,1631260,AJ Green,AJ,1610612749,MIL,25.0,73,44,29,0.603,1659.141667,182,424,0.429,155,363,0.427,22,27,0.815,18,156,174,108,40,37,7,3,157,50,541,227,1003.8,0,0,1066.0,86,62,362,164,152,231,220,353,58,77,37,364,372,180,381,227,261,215,286,235,370,99,498,290,214,62,247,281,44,225,1,0.621,0.123,114.0,107.6
2,1642358,AJ Johnson,AJ,1610612764,WAS,20.0,29,8,21,0.276,638.386667,82,213,0.385,24,90,0.267,32,37,0.865,8,51,59,76,35,12,3,18,50,27,220,-152,414.8,0,0,409.0,412,445,253,473,338,348,325,472,319,294,425,312,335,87,455,386,410,281,313,406,441,312,217,369,347,495,379,281,44,372,2,0.480,0.173,104.1,115.6
3,203932,Aaron Gordon,Aaron,1610612743,DEN,29.0,51,33,18,0.647,1446.716667,264,497,0.531,75,172,0.436,145,179,0.810,80,167,247,164,73,23,14,37,82,137,748,301,1328.4,3,0,1308.0,275,166,211,101,185,150,182,79,191,222,24,89,93,196,104,206,173,128,180,329,265,451,301,107,139,41,183,134,44,181,1,0.650,0.186,122.5,113.0
4,1628988,Aaron Holiday,Aaron,1610612745,HOU,28.0,62,39,23,0.629,792.246667,117,268,0.437,72,181,0.398,34,41,0.829,13,65,78,83,37,19,11,11,64,49,340,88,611.1,0,0,633.0,192,100,283,124,305,304,297,325,197,216,91,300,321,142,419,361,377,259,303,353,311,224,259,292,292,115,322,281,44,315,1,0.594,0.162,110.3,103.6


In [16]:
df.isnull().sum()

PLAYER_ID            0
PLAYER_NAME          0
NICKNAME             0
TEAM_ID              0
TEAM_ABBREVIATION    0
                    ..
TEAM_COUNT           0
TS_PCT               0
USG_PCT              0
OFF_RATING           0
DEF_RATING           0
Length: 71, dtype: int64

In [17]:
df_filtered = df[df['GP'] > 20]

In [33]:
df['PPG'] = (df['PTS'] / df['GP']).round(1)
df['RPG'] = (df['REB'] / df['GP']).round(1)
df['APG'] = (df['AST'] / df['GP']).round(1)

In [34]:
df.head()

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,TEAM_COUNT,TS_PCT,USG_PCT,OFF_RATING,DEF_RATING,PPG,RPG,APG
0,1630639,A.J. Lawson,A.J.,1610612761,TOR,24.0,26,14,12,0.538,486.410000,80,190,0.421,33,101,0.327,43,63,0.683,20,66,86,31,15,13,6,12,44,38,236,-17,427.7,2,0,424.0,424,364,146,239,379,352,340,388,282,281,319,281,267,419,364,359,368,397,414,397,383,241,200,327,337,299,375,159,44,364,1,0.542,0.189,109.7,111.1,9.1,3.3,1.2
1,1631260,AJ Green,AJ,1610612749,MIL,25.0,73,44,29,0.603,1659.141667,182,424,0.429,155,363,0.427,22,27,0.815,18,156,174,108,40,37,7,3,157,50,541,227,1003.8,0,0,1066.0,86,62,362,164,152,231,220,353,58,77,37,364,372,180,381,227,261,215,286,235,370,99,498,290,214,62,247,281,44,225,1,0.621,0.123,114.0,107.6,7.4,2.4,1.5
2,1642358,AJ Johnson,AJ,1610612764,WAS,20.0,29,8,21,0.276,638.386667,82,213,0.385,24,90,0.267,32,37,0.865,8,51,59,76,35,12,3,18,50,27,220,-152,414.8,0,0,409.0,412,445,253,473,338,348,325,472,319,294,425,312,335,87,455,386,410,281,313,406,441,312,217,369,347,495,379,281,44,372,2,0.480,0.173,104.1,115.6,7.6,2.0,2.6
3,203932,Aaron Gordon,Aaron,1610612743,DEN,29.0,51,33,18,0.647,1446.716667,264,497,0.531,75,172,0.436,145,179,0.810,80,167,247,164,73,23,14,37,82,137,748,301,1328.4,3,0,1308.0,275,166,211,101,185,150,182,79,191,222,24,89,93,196,104,206,173,128,180,329,265,451,301,107,139,41,183,134,44,181,1,0.650,0.186,122.5,113.0,14.7,4.8,3.2
4,1628988,Aaron Holiday,Aaron,1610612745,HOU,28.0,62,39,23,0.629,792.246667,117,268,0.437,72,181,0.398,34,41,0.829,13,65,78,83,37,19,11,11,64,49,340,88,611.1,0,0,633.0,192,100,283,124,305,304,297,325,197,216,91,300,321,142,419,361,377,259,303,353,311,224,259,292,292,115,322,281,44,315,1,0.594,0.162,110.3,103.6,5.5,1.3,1.3


In [36]:
print(f'Original Player Count: {len(df)}')
print(f'Filtered Player Counts: {len(df_filtered)}')

Original Player Count: 569
Filtered Player Counts: 447


## ED